In [3]:
import requests
import zipfile
import geopandas as gpd
import pandas as pd
import os
import shutil
from datetime import datetime
from google.colab import drive, files
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines

# Mount Google Drive
try:
    drive.mount('/content/drive')
except Exception as e:
    print(f"Drive mount error: {e}. Ensure you are running this in Google Colab.")
    exit()

# --- Constants ---
FIRMS_URL = "https://firms.modaps.eosdis.nasa.gov/data/active_fire/modis-c6.1/shapes/zips/MODIS_C6_1_South_Asia_24h.zip"

# !!! IMPORTANT: Update these paths to YOUR shapefiles in Google Drive !!!
# Shapefile for spatial intersection (e.g., detailed districts/wards)
NEPAL_DISTRICTS_INTERSECT_SHP = "/content/drive/MyDrive/ShpForColab/nepal_districts_wards.shp" # <-- UPDATE THIS
# Shapefile for plotting the map background (e.g., simpler district boundaries)
NEPAL_DISTRICTS_MAP_SHP = "/content/drive/MyDrive/ShpForColab/nepal_district.shp" # <-- UPDATE THIS
# Shapefile for protected areas
PROTECTED_AREAS_SHP = "/content/drive/MyDrive/ShpForColab/PABZ_Nov2014_FInal.shp" # <-- UPDATE THIS

# Column name in the INTERSECT shapefile that contains the district identifier
DISTRICT_COLUMN_NAME = "DISTRICT" # Adjust if necessary for your INTERSECT shapefile

DOWNLOAD_DIR = "temp_fire_data"
OUTPUT_EXCEL = f"nepal_daily_fire_report_{datetime.now().strftime('%Y%m%d')}.xlsx"
OUTPUT_MAP_IMAGE = f"nepal_daily_fire_map_{datetime.now().strftime('%Y%m%d')}.png"

# Colors based on the reference image (approximations)
DISTRICT_MAP_FILL_COLOR = '#e0f2e0'  # Pale green
DISTRICT_MAP_EDGE_COLOR = 'black'
PROTECTED_AREA_FILL_COLOR = '#8fbc8f' # Darker, muted green (DarkSeaGreen)
FIRE_POINT_COLOR = '#ff7b00'         # Orange/Red for fire
FIRE_POINT_MARKER = '*'              # Star marker, closer than 'X'

# --- Helper Functions ---

def download_file(url, target_dir):
    """Downloads a file from a URL to a target directory."""
    os.makedirs(target_dir, exist_ok=True)
    local_filename = os.path.join(target_dir, url.split('/')[-1])
    print(f"Downloading {url} to {local_filename}...")
    try:
        with requests.get(url, stream=True, timeout=60) as r:
            r.raise_for_status()
            with open(local_filename, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        print("Download complete.")
        return local_filename
    except requests.exceptions.RequestException as e:
        print(f"Download error: {e}")
        return None

def unzip_file(zip_filepath, extract_dir):
    """Unzips a file and returns the path to the first .shp file found."""
    print(f"Unzipping {zip_filepath}...")
    try:
        with zipfile.ZipFile(zip_filepath, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
        shp_files = [f for f in os.listdir(extract_dir) if f.lower().endswith('.shp')]
        if not shp_files:
            print(f"Error: No .shp file found in {extract_dir}")
            return None
        shp_path = os.path.join(extract_dir, shp_files[0])
        print(f"Shapefile found: {shp_path}")
        return shp_path
    except Exception as e:
        print(f"Unzip error: {e}")
        return None

def load_shapefile(shp_path, layer_name="layer"):
    """Loads a shapefile with error handling."""
    if not os.path.exists(shp_path):
        print(f"Warning: Shapefile not found at {shp_path}. Cannot load {layer_name}.")
        return None
    try:
        print(f"Reading {layer_name} shapefile: {shp_path}...")
        gdf = gpd.read_file(shp_path)
        print(f"{layer_name.capitalize()} shapefile loaded successfully.")
        return gdf
    except Exception as e:
        print(f"Error reading {layer_name} shapefile at {shp_path}: {e}")
        return None

def plot_fire_map(map_districts_gdf, protected_areas_gdf, fires_gdf, output_filename):
    """Generates and saves a map plot using specified colors and data."""
    print(f"Generating map...")
    fig, ax = plt.subplots(1, 1, figsize=(12, 12))
    ax.set_aspect('equal')
    ax.set_axis_off()

    # Plot MAP districts (pale green fill, black edge)
    if map_districts_gdf is not None and not map_districts_gdf.empty:
         map_districts_gdf.plot(ax=ax,
                               facecolor=DISTRICT_MAP_FILL_COLOR,
                               edgecolor=DISTRICT_MAP_EDGE_COLOR,
                               linewidth=0.5)
         district_label = 'District'
    else:
        print("Warning: No Map District data to plot.")
        district_label = 'District (not loaded)'

    # Plot protected areas (darker green fill)
    pa_label = 'Protected Areas' # Default label
    if protected_areas_gdf is not None and not protected_areas_gdf.empty:
         protected_areas_gdf.plot(ax=ax,
                                 facecolor=PROTECTED_AREA_FILL_COLOR,
                                 edgecolor='none') # No edge color in reference
    else:
         print("Warning: No Protected Areas data to plot.")
         pa_label = 'Protected Areas (not loaded)'


    # Plot fire points (orange star marker)
    fire_label = 'Fire Points' # Default label
    if fires_gdf is not None and not fires_gdf.empty:
        fires_gdf.plot(ax=ax,
                       marker=FIRE_POINT_MARKER,
                       color=FIRE_POINT_COLOR,
                       markersize=40) # Slightly larger marker
    else:
        print("No fire points detected in Nepal to plot.")
        fire_label = 'Fire Points (none detected)'

    # --- Create Custom Legend ---
    handles = []

    # Fire Point Legend Item (only if fires exist)
    if fires_gdf is not None and not fires_gdf.empty:
        fire_marker = mlines.Line2D([], [], color=FIRE_POINT_COLOR, marker=FIRE_POINT_MARKER,
                                    linestyle='None', markersize=8, label=fire_label)
        handles.append(fire_marker)

    # Protected Area Legend Item (always add, label indicates if loaded)
    pa_patch = mpatches.Patch(facecolor=PROTECTED_AREA_FILL_COLOR, edgecolor='none',
                             label=pa_label)
    handles.append(pa_patch)

    # District Legend Item (always add, label indicates if loaded)
    district_patch = mpatches.Patch(facecolor=DISTRICT_MAP_FILL_COLOR, edgecolor=DISTRICT_MAP_EDGE_COLOR,
                                   label=district_label)
    handles.append(district_patch)


    # Add legend to the plot
    ax.legend(handles=handles, loc='lower left', title="Legend", fontsize=10,
              title_fontsize=12, frameon=True, facecolor='white') # Added white background to legend

    # Add North Arrow
    ax.annotate('N', xy=(0.95, 0.95), xycoords='axes fraction', ha='center', va='center',
                fontsize=20, fontweight='bold',
                arrowprops=dict(arrowstyle="<-", facecolor='black', lw=1.5))

    # Save the figure
    try:
        plt.savefig(output_filename, dpi=300, bbox_inches='tight')
        print(f"Map successfully saved to {output_filename}")
        plt.close(fig)
        return True
    except Exception as e:
        print(f"Map saving error: {e}")
        plt.close(fig)
        return False

# --- Main Execution Logic ---

def main():
    # Download and Unzip Fire Data
    zip_path = download_file(FIRMS_URL, DOWNLOAD_DIR)
    if not zip_path: return
    fire_shp_path = unzip_file(zip_path, DOWNLOAD_DIR)
    if not fire_shp_path:
        if os.path.exists(DOWNLOAD_DIR): shutil.rmtree(DOWNLOAD_DIR)
        return

    # Load Geospatial Data using the helper function
    fire_gdf = load_shapefile(fire_shp_path, "FIRMS fire data")
    intersect_districts_gdf = load_shapefile(NEPAL_DISTRICTS_INTERSECT_SHP, "Intersection Districts")
    map_districts_gdf = load_shapefile(NEPAL_DISTRICTS_MAP_SHP, "Map Districts")
    protected_areas_gdf = load_shapefile(PROTECTED_AREAS_SHP, "Protected Areas")

    # Check if essential data for intersection was loaded
    if fire_gdf is None or intersect_districts_gdf is None:
        print("Error: Cannot proceed without FIRMS fire data and Intersection Districts data.")
        if os.path.exists(DOWNLOAD_DIR): shutil.rmtree(DOWNLOAD_DIR)
        return

    # --- CRS (Coordinate Reference System) Handling ---
    print("Checking and harmonizing Coordinate Reference Systems (CRS)...")
    # Use the MAP districts CRS as the target for plotting consistency, if available
    # Otherwise, use the INTERSECT districts CRS
    if map_districts_gdf is not None:
        target_crs = map_districts_gdf.crs
        print(f"Using target CRS from Map Districts: {target_crs}")
    else:
         target_crs = intersect_districts_gdf.crs # Fallback
         print(f"Using target CRS from Intersection Districts: {target_crs}")

    try:
        # Convert fire data
        if fire_gdf.crs != target_crs:
            print("Converting fire data CRS...")
            fire_gdf = fire_gdf.to_crs(target_crs)

        # Convert intersection districts (if needed, though unlikely if it's the source)
        if intersect_districts_gdf.crs != target_crs:
            print("Converting intersection districts CRS...")
            intersect_districts_gdf = intersect_districts_gdf.to_crs(target_crs)

        # Convert map districts (if needed and loaded)
        if map_districts_gdf is not None and map_districts_gdf.crs != target_crs:
            print("Converting map districts CRS...")
            map_districts_gdf = map_districts_gdf.to_crs(target_crs)

        # Convert protected areas (if needed and loaded)
        if protected_areas_gdf is not None and protected_areas_gdf.crs != target_crs:
             print("Converting protected areas CRS...")
             protected_areas_gdf = protected_areas_gdf.to_crs(target_crs)

        print("CRS harmonization complete.")
    except Exception as e:
        print(f"CRS conversion error: {e}")
        if os.path.exists(DOWNLOAD_DIR): shutil.rmtree(DOWNLOAD_DIR)
        return

    # --- Spatial Join using the INTERSECT districts shapefile ---
    print(f"Performing spatial join using Intersection Districts shapefile...")
    fires_in_nepal = None # Initialize
    try:
        # Ensure valid geometries
        fire_gdf['geometry'] = fire_gdf.geometry.buffer(0)
        intersect_districts_gdf['geometry'] = intersect_districts_gdf.geometry.buffer(0)

        # Perform the join with INTERSECT districts
        fires_in_nepal = gpd.sjoin(fire_gdf, intersect_districts_gdf, how='inner', predicate='within')
        print(f"Found {len(fires_in_nepal)} fire points within intersection districts.")
    except Exception as e:
        print(f"Spatial join error: {e}. Proceeding without spatial join results for the report.")
        # We can still plot the map, just the report might be empty/incomplete
        fires_in_nepal = gpd.GeoDataFrame() # Create empty GDF to avoid errors later


    # --- Generate Excel Report ---
    print("Generating Excel report...")
    fire_counts_df = pd.DataFrame(columns=["S.N.", "District", "Fire Count"]) # Default empty
    total_fires = 0

    if not fires_in_nepal.empty:
        # Check for the specified district column name in the *intersected* data
        if DISTRICT_COLUMN_NAME not in fires_in_nepal.columns:
            possible_district_cols = [col for col in fires_in_nepal.columns if 'DISTRICT' in col.upper() or 'DIST_NAME' in col.upper()]
            if possible_district_cols:
                actual_district_col = possible_district_cols[0]
                print(f"Warning: Column '{DISTRICT_COLUMN_NAME}' not found in intersected data. Using '{actual_district_col}' instead.")
            else:
                print(f"Error: Missing district column '{DISTRICT_COLUMN_NAME}' in intersected data and could not find an alternative. Report may be inaccurate.")
                actual_district_col = None # Cannot group
        else:
            actual_district_col = DISTRICT_COLUMN_NAME

        if actual_district_col: # Only proceed if we have a district column to group by
            fire_counts = fires_in_nepal.groupby(actual_district_col).size()
            fire_counts_df = fire_counts.reset_index(name='Fire Count')
            fire_counts_df = fire_counts_df.sort_values(by="Fire Count", ascending=False)
            fire_counts_df.insert(0, 'S.N.', range(1, 1 + len(fire_counts_df)))
            fire_counts_df = fire_counts_df.rename(columns={actual_district_col: "District"})
            total_fires = fire_counts_df["Fire Count"].sum()

            # Add Total row
            total_row = pd.DataFrame([{"S.N.": "", "District": "Total", "Fire Count": total_fires}])
            fire_counts_df = pd.concat([fire_counts_df, total_row], ignore_index=True)
        else:
             print("Cannot generate district counts due to missing column.")

    else:
        print("No fires detected within intersection districts for the report.")
        # Add Total row with 0 count to empty df
        total_row = pd.DataFrame([{"S.N.": "", "District": "Total", "Fire Count": 0}])
        fire_counts_df = pd.concat([fire_counts_df, total_row], ignore_index=True)


    # --- Generate Map Visualization using the MAP districts shapefile ---
    # Pass the correct GDFs: map_districts for background, PAs, and the fires found within Nepal boundaries
    map_saved = plot_fire_map(
        map_districts_gdf,       # Use the MAP districts for plotting
        protected_areas_gdf,
        fires_in_nepal,          # Plot the points found within the INTERSECT districts
        OUTPUT_MAP_IMAGE
    )

    # --- Export and Download Files ---
    print("Exporting Excel report and downloading files...")
    try:
        fire_counts_df.to_excel(OUTPUT_EXCEL, index=False, engine='openpyxl')
        print(f"Excel report saved to {OUTPUT_EXCEL}")
        files.download(OUTPUT_EXCEL)
        print(f"Excel report download initiated.")

        if map_saved:
            files.download(OUTPUT_MAP_IMAGE)
            print(f"Map image download initiated.")

    except Exception as e:
        print(f"File export or download error: {e}")

    # --- Clean up ---
    print("Cleaning up temporary files...")
    try:
        if os.path.exists(DOWNLOAD_DIR):
            shutil.rmtree(DOWNLOAD_DIR)
            print(f"Removed temporary directory: {DOWNLOAD_DIR}")
    except Exception as e:
        print(f"Cleanup warning: Could not remove {DOWNLOAD_DIR}. {e}")

    print("Processing finished.")

# --- Run the main function ---
if __name__ == "__main__":
    main()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Download complete.
Unzipping temp_fire_data/MODIS_C6_1_South_Asia_24h.zip...
Shapefile found: temp_fire_data/MODIS_C6_1_South_Asia_24h.shp
Reading FIRMS fire data shapefile: temp_fire_data/MODIS_C6_1_South_Asia_24h.shp...
Firms fire data shapefile loaded successfully.
Reading Intersection Districts shapefile: /content/drive/MyDrive/ShpForColab/nepal_districts_wards.shp...
Intersection districts shapefile loaded successfully.
Reading Map Districts shapefile: /content/drive/MyDrive/ShpForColab/nepal_district.shp...
Map districts shapefile loaded successfully.
Reading Protected Areas shapefile: /content/drive/MyDrive/ShpForColab/PABZ_Nov2014_FInal.shp...
Protected areas shapefile loaded successfully.
Checking and harmonizing Coordinate Reference Systems (CRS)...
Using target CRS from Map Districts: EPSG:4326
CRS harmonization complete.
Performing spatial join us

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Excel report download initiated.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Map image download initiated.
Cleaning up temporary files...
Removed temporary directory: temp_fire_data
Processing finished.
